# Create Network

In this repository, we will create a coauthorship network with the researchers in our database.

In [22]:
%matplotlib inline
import sqlite3
import pandas as pd
import itertools
import networkx as nx
from unicodedata import normalize
import json

In [2]:
authors =  pd.read_csv('./views/Authors.csv',encoding='latin-1',sep=';',index_col=0)
papers  =  pd.read_csv('./views/Papers.csv',encoding='latin-1',sep=';',index_col=0)

In [3]:
authors = authors.reset_index()
authors.head(3)

,index,@DATA-ATUALIZACAO,@NUMERO-IDENTIFICADOR,_id,NOME-COMPLETO,NOME-INSTITUICAO,UF-INSTITUICAO,PAIS-INSTITUICAO,PAIS,NOME-CITACOES,CODIGO-INSTITUICAO
0,0,2014-10-07,2.558570e+15,5d31b6f7a4ef2aef4f143ec2,Aarão Lyra,Universidade Potiguar,RN,Brasil,Brasil,"LYRA, A.;Lyra, A.;Lyra, Aarão;LYRA, A",000100000991
1,1,2017-08-01,3.172488e+15,5d31bcc1a4ef2aef4f144f75,Abdelhakim Senhaji Hafid,NaN,NaN,NaN,Marrocos,"HAFID, A. S.;A. HAFID;HAFID, ABDELHAKIM;HAFID,...",NaN
2,2,2019-09-04,2.745843e+15,5d31b766a4ef2aef4f14451d,Abdelmoubine Amar Henni,Universidade Federal de Santa Catarina,SC,Brasil,Argélia,"HENNI, A. A.;HENNI, ABDELMOUBINE AMAR;Amar Abd...",004300000009


In [4]:
papers.head(3)


,@NUMERO-IDENTIFICADOR,_id,NOME-COMPLETO,TITULO-DO-ARTIGO,ANO-DO-ARTIGO,JOURNAL,PALAVRAS-CHAVE-ARTIGO,COAUTORES,CHAVE_ARTIGO
0,6.115399e+15,5d31b5b9a4ef2aef4f14294d,Manuel Francisco Zuloeta Jimenez,"Weak almost periodic motions, minimality and s...",2014,Journal of Differential Equations (Print),Sistemas semidinâmicos Impulsivos|Weak almost ...,"[{'@NOME-COMPLETO-DO-AUTOR': 'BONOTTO, E.M.', ...",WEAKALMOSTPERIODICMOTIONSMINIMALITYANDSTABILIT...
1,6.115399e+15,5d31b5b9a4ef2aef4f14294d,Manuel Francisco Zuloeta Jimenez,"On impulsive semidynamical systems: minimal, r...",2014,Topological Methods in Nonlinear Analysis,Semidynamical systems|Minimal sets|Recurrence|...,"[{'@NOME-COMPLETO-DO-AUTOR': 'BONOTTO, E.M.', ...",ONIMPULSIVESEMIDYNAMICALSYSTEMSMINIMALRECURREN...
2,6.115399e+15,5d31b5b9a4ef2aef4f14294d,Manuel Francisco Zuloeta Jimenez,Negative trajectories in impulsive semidynamic...,2015,Journal of Differential Equations (Print),Impulsive semidinâmical systems|Negative semi ...,"[{'@NOME-COMPLETO-DO-AUTOR': 'AFONSO, S.M.', '...",NEGATIVETRAJECTORIESINIMPULSIVESEMIDYNAMICALSY...


# Creating sqlite database

In [5]:
author_paper_ids = papers[['NOME-COMPLETO','CHAVE_ARTIGO']].groupby(by=['NOME-COMPLETO',
                                                                        'CHAVE_ARTIGO']).count().reset_index()

In [7]:
schema = """
DROP TABLE IF EXISTS "author_paper";
CREATE TABLE IF NOT EXISTS "author_paper" (
    paper_id VARCHAR,
    author_id VARCHAR,
    FOREIGN KEY(paper_id) REFERENCES paper(id)
    FOREIGN KEY(author_id) REFERENCES author(id)
);
"""

In [8]:
conn = sqlite3.connect('pbmnet.sqlite',timeout=10)
cursor = conn.cursor()
cursor.executescript(schema)
conn.commit()
conn.close()

In [9]:
ins_paper_author ="""
INSERT INTO author_paper (paper_id, author_id) \
    VALUES (?,?);
"""

conn = sqlite3.connect('pbmnet.sqlite',timeout=10)
for index,row in author_paper_ids.iterrows():
    conn.execute(ins_paper_author,[row['CHAVE_ARTIGO'],row['NOME-COMPLETO']])
conn.commit()
conn.close()

# Creating edge list

In [10]:
def Generate_Edges_Authors(db):
    DATABASE = db
# Cria lista de arestas para rede de autores
    query_pair_authors = """
    SELECT paper_id,
           a1.author_id AS author1,
           a2.author_id AS author2
    FROM author_paper AS a1
    JOIN author_paper AS a2 USING (paper_id)
    WHERE a1.author_id < a2.author_id;
    """
    conn = sqlite3.connect(DATABASE,timeout=10)
    edges_authors = []
    for row in conn.execute(query_pair_authors):
        edges_authors.append(row)
    conn.close()
    edges_authors= pd.DataFrame(edges_authors,dtype=int,columns=['paper','author1',
                                                                 'author2']).applymap(lambda x: int(x))
    
    count_authors = edges_authors.groupby(['author1','author2']).count().reset_index()
    return count_authors

In [11]:
DATABASE = 'pbmnet.sqlite'
# Cria lista de arestas para rede de autores
query_pair_authors = """
SELECT paper_id,
       a1.author_id AS author1,
       a2.author_id AS author2
FROM author_paper AS a1
JOIN author_paper AS a2 USING (paper_id)
WHERE a1.author_id < a2.author_id;
"""
conn = sqlite3.connect(DATABASE,timeout=10)
edges_authors = []
for row in conn.execute(query_pair_authors):
    edges_authors.append(row)
conn.close()

edges_authors= pd.DataFrame(edges_authors,columns=['paper','author1','author2'])

In [12]:
count_authors = edges_authors.groupby(['author1','author2']).count().reset_index()

In [13]:
count_authors = pd.merge(count_authors,authors[['index','NOME-COMPLETO']],
                             left_on='author1',right_on='NOME-COMPLETO',how='left')

count_authors = pd.merge(count_authors,authors[['index','NOME-COMPLETO']],
                             left_on='author2',right_on='NOME-COMPLETO',how='left',suffixes=('1','2'))

In [14]:
count_authors = count_authors[['index1','index2','paper']]
count_authors.columns = ['author1','author2','paper']

In [15]:
count_authors.head()

,author1,author2,paper
0,0,1413,4
1,0,9846,1
2,2,7297,3
3,4,296,1
4,4,1096,1


In [27]:
authors_dict = authors[['index','NOME-COMPLETO','NOME-INSTITUICAO','UF-INSTITUICAO',
                         'PAIS-INSTITUICAO','PAIS',]].to_dict(orient='records')

# Creating the Network

In [28]:
def Create_Network(nodes,edges,source='source', target='target',weight='weight'):
    G = nx.Graph()
    
    for d in nodes:
        node_id = int(d['index'])
        G.add_node(node_id, **d) 
    
    for d in edges.to_dict('records'):
        G.add_edge(d[source],d[target],count = d[weight])
    
#     pos = nx.spring_layout(G,k=2)
#     pos = pd.DataFrame(pos).transpose()
    
#     pos['id'] = pos.index
#     pos.columns = ['x','y','id']

    
    return G

In [29]:
G_authors = Create_Network(authors_dict,edges_authors,'author1','author2','paper')

In [33]:
nx.draw_spring(G_authors)

MemoryError: 